#### rules

- L : 롤러코스터 탑승 가능 인원
- C : 롤러코스터 운행 횟수
- N : 대기하는 손님 그룹 수
- Pi : 각 그룹의 사람 수

- 그룹은 반드시 함께 탑승 (좌석이 빈 상태로 운행 가능)

- 롤러코스터를 탄 인원은 줄의 맨 뒤로 이동

In [10]:
# 입력 받기

# l(탑승 정원), c(운행횟수), n(그룹 수) 공백으로 입력 받기
l, c, n = [int(i) for i in input().split()]

# 각 그룹의 인원수 입력 받기
groups = [int(input()) for i in range(n)]


print('탑승인원 L:{0}, 운행횟수 C:{1}, 그룹 수 N:{2}'.format(l,c,n))
print('각 그룹 인원수',groups)

# total
total_revenue = 0

5 3 2
2
3
탑승인원 L:5, 운행횟수 C:3, 그룹 수 N:2
각 그룹 인원수 [2, 3]


In [11]:
# 롤러코스터 운행

for i in range(c):       # 운행횟수(c) 만큼 운행 반복
    num_passengers = 0   # 롤러코스터 탑승 인원수 -> L과 비교
    people_on_ride = []  # 롤러코스터 탑승객을 담기 위한 배열
    
    while len(groups) > 0 and num_passengers + groups[0] <= l:
        
        group = groups.pop(0)
        people_on_ride.append(group)
        num_passengers += group
    
    total_revenue += num_passengers
    groups.extend(people_on_ride)

print(total_revenue)

# 해당 코드의 문제점
# 엄청나게 큰 데이터 셋 => time out

15


#### sol 1

+ 1번 그룹이 나가면 다시 맨 뒤에 1번 그룹이 붙는 구조 => 원형 큐
+ 만약 10개의 그룹에 있다면 그룹에게 0 ~ 9번 까지의 번호표를 부여하는 것 과 동일하다.
+ 1번 그룹이 빠져나가면 즉, 0번 번호표를 처리했으니 다음은 1번 번호표를 처리하면 된다.

+ 즉, 이를 이용하여 그룹 멤버를 직접 옮기지 않고 번호표만 옮기면 속도를 개선할 수 있을 것이다.
+ 다음에 탑승할 순번을 가리킬 인덱스 생성

In [12]:
l,c,n = [int(i) for i in input().split()]
groups = [int(input()) for i in range(n)]

total_revenue = 0
next_idx = 0       # idx를 이용하여 다음에 탑승할 손님을 가리키도록 한다.

for i in range(c):
    num_passengers = 0
    
    while num_passengers + groups[next_idx] <= l:
        num_passengers += groups[next_idx]
        next_idx = (next_idx + 1) % n    # 그룹 수 만큼 원형 큐 인덱스 변경
        
    total_revenue += num_passengers

print(total_revenue)

# 문제점
# 이전 방법은 배열을 통해서 넣고 빼고 했기 때문에 남아있는 손님 확인이 용이하였다.
# idx를 사용할 경우 배열을 비우는 행위가 없기 때문에 남은 인원을 확인하기 어려움
# 따라서 시작 idx를 저장할 변수 하나 더 생성


5 3 2
2
3
15


In [14]:
# sol 1 + 처음 순서를 가리키는 인덱스 생성

l,c,n = [int(i) for i in input().split()]
groups = [int(input()) for i in range(n)]

total_revenue = 0
next_idx = 0       # idx를 이용하여 다음에 탑승할 손님을 가리키도록 한다.

for i in range(c):
    num_passengers = 0
    begin_idx = next_idx    # 운행 시작 시 맨 처음 탑승할 인덱스 저장
    
    while num_passengers + groups[next_idx] <= l:
        num_passengers += groups[next_idx]
        next_idx = (next_idx + 1) % n    # 그룹 수 만큼 원형 큐 인덱스 변경
        
        if begin_idx == next_idx: # 이미 탑승한 사람이 또 탑승할 상황이 발생하면 반복문 탈출
            break
        
    total_revenue += num_passengers

print(total_revenue)


5 3 2
2
3
15


In [15]:
# sol 1 - 2
# begin idx 없이 구하기
# 맨 처음에 total_passengers인원과 롤러코스터 탑승 정원을 구한다.
# 만약에 롤러코스터 탑승 정원이 더 많다면 모든 사람들이 운행횟수 만큼 탈 수 있음을 의미한다.


l,c,n = [int(i) for i in input().split()]
groups = [int(input()) for i in range(n)]

total_revenue = 0
total_passengers = sum(groups)

if total_passengers <= l:
    total_revenue = total_passengers * c

else:
    next_idx = 0
    
    for i in range(c):
        num_passengers = 0
        
        while num_passengers + groups[next_idx] <= l:
            num_passengers += groups[next_idx]
            next_idx = (next_idx + 1) % n    # 그룹 수 만큼 원형 큐 인덱스 변경
            
        total_revenue += num_passengers

print(total_revenue)


5 3 2
2
3
15


+ 하지만 타임아웃 발생
+ 탑승 횟수가 매우 많아 승객이 여러번 탑승하는 것이 핵심
+ 손님이 수십 번, 수백 번 반복해서 탑승하다 보면 어느 순간 '같은 패턴'이 반복되는 경우 있음
+ 즉 해당 패턴이 발견되면 이전에 계산한 값을 저장하여 별도의 계산없이 바로 적용

+ 문제 해결 중간 과정에서 얻은 값을 재사용하여 전체 문제의 계산을 빠르게 수행하도록 코드 작성 => 동적 프로그래밍

In [18]:
# sol 2
# while 반복문을 빠져나오면
# 탑승 인원 & 탑승 인덱스만 변화

# 딕셔너리를 이용하여 => 시작 인덱스에 대응하는 탑승인원 & 탑승 인덱스 값 저장
# 따라서 다음 탑승 인덱스가 n 일떼, dic[n] 값이 있다면 => 별도의 계산 없이 사용


l,c,n = [int(i) for i in input().split()]
groups = [int(input()) for i in range(n)]

total_revenue = 0
total_passengers = sum(groups)

if total_passengers <= l:
    total_revenue = total_passengers * c

else:
    revenue_history = {}    # 시작인덱스 -> 다음 인덱스 & 탑승 인원 딕셔너리 생성
    next_idx = 0
    
    for i in range(c):
        num_passengers = 0
        begin_idx = next_idx
        
        while num_passengers + groups[next_idx] <= l:
            num_passengers += groups[next_idx]
            next_idx = (next_idx + 1) % n    # 그룹 수 만큼 원형 큐 인덱스 변경
            
        revenue_history[begin_idx] = (next_idx, num_passengers)    # 손님 탑승이 완료 되었을 때 (while 탈출) -> 정보 저장
        total_revenue += num_passengers

print(total_revenue)


5 3 2
2
2
12


In [ ]:
# sol 2
# while 반복문을 빠져나오면
# 탑승 인원 & 탑승 인덱스만 변화

# 딕셔너리를 이용하여 => 시작 인덱스에 대응하는 탑승인원 & 탑승 인덱스 값 저장
# 따라서 다음 탑승 인덱스가 n 일떼, dic[n] 값이 있다면 => 별도의 계산 없이 사용


l,c,n = [int(i) for i in input().split()]
groups = [int(input()) for i in range(n)]

total_revenue = 0
total_passengers = sum(groups)

if total_passengers <= l:
    total_revenue = total_passengers * c

else:
    revenue_history = {}    # 시작인덱스 -> 다음 인덱스 & 탑승 인원 딕셔너리 생성
    next_idx = 0
    
    # 탑승 시작
    for i in range(c):
        num_passengers = 0    # 탑승 인원 수 0 초기화
        
        # case 1 탑승 정보가 딕셔너리에 있는 경우 => next_idx와 탑승 가능 인원 수 바로 결정
        if next_idx in revenue_history:
            next_idx, num_passengers = revenue_history[next_idx]
        
        # case 2 딕셔너리에 없다면 => 직접 계산
        else:
            begin_idx = next_idx

            while num_passengers + groups[next_idx] <= l:
                num_passengers += groups[next_idx]
                next_idx = (next_idx + 1) % n    # 그룹 수 만큼 원형 큐 인덱스 변경
            
            # 해당 정보를 다음에 사용할 수 있도록
            revenue_history[begin_idx] = (next_idx, num_passengers)    # 손님 탑승이 완료 되었을 때 (while 탈출) -> 정보 저장
            
        total_revenue += num_passengers

print(total_revenue)